In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Final Waveset
This notebook contains the studies for the final waveset that produces the main result of the thesis

Fit Details:
* Only spin-1 ($J=1$) waves
  * {$J^P\ell$} = {$1^+S, 1^-P, 1^+D$}
  * Waves all unconstrained, no D/S ratio or phase parameters
* No isotropic background term
* Uses all 4 polarization orientations
* Split data into bins of mass and $-t$
  * 20 MeV wide bins from $1.0 < M_{\omega\pi^0} < 1.8$ 
* Perform 500 randomized fits in each bin of mass and $-t$ independently
  * Additional 100 bootstrap samples in each bin to improve errors and study correlations

## Setup

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
from uncertainties import ufloat, unumpy
import matplotlib.pyplot as plt

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/data-fits/spin-1/no-iso-unconstrained/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
# load in the preprocessed results 
t_bins = ["t_0.10-0.16", "t_0.16-0.23", "t_0.23-0.35", "t_0.35-1.00"]
binned_results = {}
for t_bin in t_bins:
    with open(f"{STUDY_DIR}/{t_bin}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
        data = pkl.load(f)
        binned_results[t_bin] = ResultManager(**data)

binned_results: dict[str, ResultManager]

In [ ]:
for t_bin, results in binned_results.items():
    print(f"Results for t bin: {t_bin}")
    results.summary()

## Standard Plots
First we'll take a look at the typical plots that describe the general features of the model

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(25, 5))
for i, (t_bin, results) in enumerate(binned_results.items()):    
    results.plot.intensity.jp(ax=axs[i])
    t_low, t_high = results.get_t_edges()    
    axs[i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV²", fontsize=16)
    if i != 0:
        axs[i].set_ylabel("")
        axs[i].get_legend().remove()
plt.savefig(f"{STUDY_DIR}/jp_all_t_bins.pdf")    

The below plots can't be aggregated easily across t bins, so just plot for each t bin. They're already saved in the study dir, so just plt.show() them here

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves()
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)    
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.intensity.waves(fractional=True)
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)    
    plt.show()

In [ ]:
for t_bin, results in binned_results.items():
    results.plot.diagnostic.matrix()
    t_low, t_high = results.get_t_edges()
    utils.big_print(rf"{t_low} < -t < {t_high} GeV²", 2.0)    
    plt.show()

The set of all moments in each t bin is already saved, but lets look at just the significant ones across the t bins

In [ ]:
sig_moments = set()
for t_bin, results in binned_results.items():    
    sig_moments = sig_moments.union(results.get_significant_moments(threshold=0.02))

fig, axs = plt.subplots(
    len(sig_moments), 4, figsize=(20, 25), layout="constrained", sharex=True, sharey="row"
)

row_max = {}
bin_width = 0
for col, (t_bin, results) in enumerate(binned_results.items()):    
    for row, moment in enumerate(sig_moments):        
        ax = axs[row, col]
        
        my_kwargs = {moment: {"label": ""}}
        results.plot.intensity.plot([moment], ax=ax, col_kwargs=my_kwargs)        

        # only put y-label on the first column, and only put the moment name there
        if col != 0:
            ax.set_ylabel("")
        else:
            ax.set_ylabel(utils.convert_moment_name(moment))

        # only put x-label on the last row
        if row != len(sig_moments) - 1:
            ax.set_xlabel("")
        else:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")

        # track max value for each row to set the same y-lims across columns
        max_val = ax.get_ylim()[1]
        if row not in row_max or max_val > row_max[row]:
            row_max[row] = max_val
    
    # label first row with t-bin info
    t_low, t_high = results.get_t_edges()
    axs[0,col].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$")

    # get for y title
    bin_width = results.get_mass_bin_width()

# set the same y-lims across columns for each row, and save the figure
for col, (t_bin, results) in enumerate(binned_results.items()):
    for row, moment in enumerate(sig_moments):
        axs[row, col].set_ylim(top=row_max[row])              


fig.supylabel(f"Moment Value / {bin_width:.3f} GeV", fontsize=20)

plt.savefig(f"{STUDY_DIR}/significant_moments_all_t_bins.pdf")

## D/S Ratio and Phase

Even though the ratio and phase are not constrained like in past analyses, we can plot the phase difference, and calculate the ratio between the individual wave's amplitude.

In [ ]:
# we only need to track S-waves to remove, since it will remove partner D-wave as well
all_s_waves = [c for c in binned_results[t_bins[0]].coherent_sums["eJPmL"] if c.endswith("S")]

for s_wave in all_s_waves:
    exclude_waves = all_s_waves.copy()
    exclude_waves.remove(s_wave) # we want to keep one DS wave at a time

    fig, axs = plt.subplots(
        3,
        4,    
        sharey="row",
        figsize=(20, 10),
        gridspec_kw={"wspace": 0.0, "hspace": 0.12},
        layout="constrained",
    )
    for i, (t_bin, results) in enumerate(binned_results.items()):
        results.plot.diagnostic.ds_ratio(axs = axs[:, i], exclude_waves=exclude_waves)
        if s_wave.startswith("p"):            
            axs[0,i].set_yscale("linear")
            axs[0,i].set_ylim(0, 1)
        t_low, t_high = results.get_t_edges()
        axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=16)
        if i != 0:
            for ax in axs[:, i]:
                if ax.get_legend() is not None:
                    ax.get_legend().remove()
    plt.savefig(f"{STUDY_DIR}/ds_ratio_{s_wave}.pdf")

Lets calculate the average ratio and phase for each D/S pair in the same E852 range, and plot it as a function of -t for each pair

In [ ]:
ds_pairs_pos = [["p1pmS", "p1pmD"], ["p1p0S", "p1p0D"], ["p1ppS", "p1ppD"]]

# plot of 2 params (ratio, phase) by 3 columns (DS wave pairs) as function of t in each plot
fig, axs = plt.subplots(
    2, 3,
    sharex=True,
    sharey="row",
    figsize=(15, 7),
    gridspec_kw={"wspace": 0.0, "hspace": 0.1},
    layout="constrained",
)

marker_list = ["v", "o", "^"]
for col, (s_wave, d_wave) in enumerate(ds_pairs_pos):
    # calculate the average ds ratio and its standard deviation for the E852 mass range in each t bin
    avg_ratios = []
    avg_ratio_errs = []
    avg_phases = []
    avg_phases_errs = []
    t_avg = []
    t_rms = []
    for i, (t_bin, results) in enumerate(binned_results.items()):        
        indices = results.get_fit_indices(1.155, 1.315) # E852 mass range

        # ratio
        s_wave_values = results.fit_df.loc[indices, s_wave]
        d_wave_values = results.fit_df.loc[indices, d_wave]

        ratio_values = np.sqrt(d_wave_values / s_wave_values)        
        avg_ratio = np.mean(ratio_values)
        ratio_err = np.std(ratio_values)
        avg_ratios.append(avg_ratio)
        avg_ratio_errs.append(ratio_err)

        # phase
        phase_values = results.fit_df.loc[indices, results.phase_difference_dict[(s_wave, d_wave)]]
        avg_phase = np.mean(phase_values)
        phase_err = np.std(phase_values)
        avg_phases.append(avg_phase)
        avg_phases_errs.append(phase_err)

        # get the average of RMS and t averages in the mass range
        t_avg_values = results.data_df.loc[indices, "t_avg"]
        t_rms_values = results.data_df.loc[indices, "t_rms"]
        t_avg.append(np.mean(t_avg_values))
        t_rms.append(np.mean(t_rms_values))

    # now plot the average ratios and phases as a function of t, with error bars
    label = utils.convert_amp_name(s_wave).replace("S", "(D/S)")
    axs[0, col].errorbar(
        x=t_avg, xerr=t_rms, y=avg_ratios, yerr=avg_ratio_errs, 
        marker=marker_list[col], linestyle="", color="red", markersize=6
    )
    axs[1, col].errorbar(
        x=t_avg, xerr=t_rms, y=avg_phases, yerr=avg_phases_errs,
        marker=marker_list[col], linestyle="", color="red", markersize=6
    )
    axs[1, col].errorbar(
        x=t_avg, xerr=t_rms, y=np.array(avg_phases)*-1, yerr=avg_phases_errs,
        marker=marker_list[col], linestyle="", color="red", markersize=6
    )

    # plot the e852 ratio and phase, using t bin midpoint and range as x, xerr
    axs[0, col].errorbar(
        x=[0.8], xerr=[0.7], y=[0.269], yerr=[0.019], 
        marker=".", linestyle="", label="E852", color="black",
    )
    axs[1, col].errorbar(
        x=[0.8], xerr=[0.7], y=[10.54], yerr=[6.3],
        marker=".", linestyle="", label="E852", color="black",
    )

    axs[0, col].set_xlim(0, 1.0)
    axs[1, col].set_xlim(0, 1.0)

    axs[0, col].set_title(rf"{label}", fontsize=16)

    axs[0, col].set_ylim(bottom=0)
    axs[1,col].set_ylim(-180, 180)
    axs[1,col].set_yticks(np.linspace(-180, 180, 5))

    axs[1,col].set_xlabel(r"-t (GeV$^2$)")

    if col == 0:
        axs[0, col].set_ylabel("D/S Ratio", loc="top")
        axs[1, col].set_ylabel("D-S Phase (°)", loc="top")
        axs[0,col].legend()
        axs[1,col].legend()

plt.savefig(f"{STUDY_DIR}/ds_vs_t_pos.pdf")

## $b_1(1235)$ and $1^{--}$ interference

In [ ]:
fig, axs = plt.subplots(
    2,
    4,
    sharex=True,   
    sharey="row",     
    gridspec_kw={"wspace": 0.0, "hspace": 0.07},
    height_ratios=[3, 1],
    figsize=(20, 8),
    layout="constrained",
)

for i, (t_bin, results)in enumerate(binned_results.items()):
    colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot

    results.plot.phase.mass_phase(
        amp1="p1p0S", amp2="p1mpP",
        amp1_kwargs={"color":colors[2], "alpha": 1.0},
        amp2_kwargs={"color":colors[3], "alpha": 1.0},
        amp_ax=axs[0, i],
        phase_ax=axs[1, i],
    )
    t_low, t_high = results.get_t_edges()
    axs[0,i].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=18)    
    if i != 0:
        axs[0,i].set_ylabel("")
        axs[1,i].set_ylabel("")
        axs[0,i].get_legend().remove()

for i in range(4):
    max_val = axs[0,i].set_ylim(top=7e5)

plt.savefig(f"{STUDY_DIR}/mass_phase_all_t_bins.pdf")

Let's view the bootstrap distributions with a joyplot

In [ ]:
# they have to be made individually because the joyplot is a bunch of axes, so we can't easily put them all in one figure
for i, (t_bin, results)in enumerate(binned_results.items()):
    t_low, t_high = results.get_t_edges()
    utils.big_print(f"{t_low} < -t < {t_high}", 2.0)
    results.plot.bootstrap.joyplot(
        ["p1p0S", "p1mpP"]        
    )
    plt.savefig(f"{STUDY_DIR}/{t_bin}/plots/joyplot_p1p0S_p1mpP.pdf")
    plt.show()

## Naturality Contributions by $J^P$

## Correlations
* Plot correlations as function of mass, all t bins on same plot diff linestyles, looking at strongly correlated waves
  * m1p0D -> p1p0D

## Breit-Wigner Lineshape Fits

In [ ]:
from scipy.optimize import curve_fit
from neutralb1.analysis.physics import breit_wigner
vec_bw = np.vectorize(breit_wigner)

# normal breit-wigner, with a tunable scale factor.
# since this Breit-Wigner is defined typically at the amplitude level, we want to view its intensity, so we take |BW|^2
def s_wave_bw_model(mass, scale, bw_mass, bw_width):
    bw = vec_bw(mass, bw_mass=bw_mass, bw_width=bw_width, bw_l=0)    
    return scale * np.square(np.abs(bw))
def p_wave_bw_model(mass, scale, bw_mass, bw_width):
    bw = vec_bw(mass, bw_mass=bw_mass, bw_width=bw_width, bw_l=1)    
    return scale * np.square(np.abs(bw))
def d_wave_bw_model(mass, scale, bw_mass, bw_width):
    bw = vec_bw(mass, bw_mass=bw_mass, bw_width=bw_width, bw_l=2)    
    return scale * np.square(np.abs(bw))

### S-wave

In [ ]:
function_masses = np.arange(1.0, 1.5, 0.001) # centered on b1 region
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharex=True
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_y_titles = [r"$1^+S_{+1}^{(+)}$", r"$1^+S_{0}^{(+)}$", r"$1^+S_{-1}^{(+)}$"]
    row_wave = ["p1ppS", "p1p0S", "p1pmS"]
    for row in range(3):                
        data_masses = results.get_mass_centers()
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        # determine where data_mass > 1.5 GeV
        fit_masses = [m for m in data_masses if m < 1.5]
        fit_values = data_values[:len(fit_masses)] 
        fit_errors = data_errors[:len(fit_masses)]       

        popt, pcov = curve_fit(
            s_wave_bw_model,
            fit_masses,
            fit_values,
            sigma=fit_errors,
            p0=[200000, 1.23, 0.14],
        )
        perr = np.sqrt(np.diag(pcov))
        scale_opt, mass_opt, width_opt = popt
        fitted_curve = s_wave_bw_model(function_masses, *popt)

        ax = axs[row, col]
        ax.plot(function_masses, fitted_curve, color="red", linestyle="-", linewidth=2, label="Fitted BW")
        ax.errorbar(
            x=data_masses, 
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker=".", 
            color="black",             
        )
        
        ax.set_ylabel(row_y_titles[row] if col == 0 else "", loc="center", fontsize=18)
        ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

        # Plot PDG shape for comparison
        pdg_mass = 1.2295
        pdg_width = 0.142
        pdg_curve = s_wave_bw_model(function_masses, scale_opt, pdg_mass, pdg_width)
        ax.plot(function_masses, pdg_curve, color="blue", linestyle="--", linewidth=2, alpha=0.6, label="PDG BW")

        # calculate chi squared per degree of freedom for the fit
        residuals = fit_values - s_wave_bw_model(fit_masses, *popt)
        chi_squared = np.sum((residuals / fit_errors) ** 2)
        dof = len(fit_values) - len(popt)
        chi_squared_per_dof = chi_squared / dof
        
        if row == 0:
            t_low, t_high = results.get_t_edges()
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=16)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
        
        textstr = (
            f"M: {round(mass_opt*1000)} ± {round(perr[1]*1000)} MeV\n"
            rf"$\Gamma$: {round(width_opt*1000)} ± {round(perr[2]*1000)} MeV"
            "\n"
            rf"$\chi^2$/df: {chi_squared_per_dof:.2f}"
        )
        ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=12,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.savefig(f"{STUDY_DIR}/BW-b1_s_refl+_all_t_bins.pdf")

### P-wave

In [ ]:
function_masses = np.arange(1.0, 1.8, 0.001) # centered on b1 region
fig, axs = plt.subplots(
    1, 4, figsize=(20, 6), layout="constrained", sharey=True, sharex=True
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    row_y_titles = [r"$1^-P_{+1}^{(+)}$"]
    row_wave = ["p1mpP"]
    for row in range(1):                
        data_masses = results.get_mass_centers()
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values
        
        fit_masses = data_masses
        fit_values = data_values
        fit_errors = data_errors

        popt, pcov = curve_fit(
            p_wave_bw_model,
            fit_masses,
            fit_values,
            sigma=fit_errors,
            p0=[150000, 1.465, 0.4],
        )
        perr = np.sqrt(np.diag(pcov))
        scale_opt, mass_opt, width_opt = popt
        fitted_curve = p_wave_bw_model(function_masses, *popt)

        ax = axs[col]
        ax.plot(function_masses, fitted_curve, color="red", linestyle="-", linewidth=2, label="Fitted BW")
        ax.errorbar(
            x=data_masses, 
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker=".", 
            color="black",             
        )
        
        ax.set_ylabel(row_y_titles[row] if col == 0 else "", loc="center", fontsize=18)
        ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

        # Plot PDG shape for comparison
        pdg_mass = 1.465
        pdg_width = 0.400
        pdg_curve = p_wave_bw_model(function_masses, scale_opt, pdg_mass, pdg_width)
        ax.plot(function_masses, pdg_curve, color="blue", linestyle="--", linewidth=2, alpha=0.6, label="PDG BW")

        # calculate chi squared per degree of freedom for the fit
        residuals = fit_values - p_wave_bw_model(fit_masses, *popt)
        chi_squared = np.sum((residuals / fit_errors) ** 2)
        dof = len(fit_values) - len(popt)
        chi_squared_per_dof = chi_squared / dof
        
        t_low, t_high = results.get_t_edges()
        ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=16)    

        ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
        
        textstr = (
            f"M: {round(mass_opt*1000)} ± {round(perr[1]*1000)} MeV\n"
            rf"$\Gamma$: {round(width_opt*1000)} ± {round(perr[2]*1000)} MeV"
            "\n"
            rf"$\chi^2$/df: {chi_squared_per_dof:.2f}"
        )
        ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=12,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.savefig(f"{STUDY_DIR}/BW-b1_p_refl+_all_t_bins.pdf")

### D-wave

In [ ]:
function_masses = np.arange(1.0, 1.5, 0.001) # centered on b1 region
fig, axs = plt.subplots(
    3, 4, figsize=(20, 10), layout="constrained", sharex=True
)

for col, (t_bin, results) in enumerate(binned_results.items()):
    row_y_titles = [r"$1^+D_{+1}^{(+)}$", r"$1^+D_{0}^{(+)}$", r"$1^+D_{-1}^{(+)}$"]
    row_wave = ["p1ppD", "p1p0D", "p1pmD"]
    for row in range(3):                
        data_masses = results.get_mass_centers()
        data_values = results.fit_df[row_wave[row]].values
        data_errors = results.fit_df[f"{row_wave[row]}_err"].values

        # determine where data_mass > 1.5 GeV
        fit_masses = [m for m in data_masses if m < 1.5]
        fit_values = data_values[:len(fit_masses)] 
        fit_errors = data_errors[:len(fit_masses)]       

        popt, pcov = curve_fit(
            d_wave_bw_model,
            fit_masses,
            fit_values,
            sigma=fit_errors,
            p0=[200000, 1.23, 0.14],
        )
        perr = np.sqrt(np.diag(pcov))
        scale_opt, mass_opt, width_opt = popt
        fitted_curve = d_wave_bw_model(function_masses, *popt)

        ax = axs[row, col]
        ax.plot(function_masses, fitted_curve, color="red", linestyle="-", linewidth=2, label="Fitted BW")
        ax.errorbar(
            x=data_masses, 
            y=data_values, 
            yerr=data_errors, 
            linestyle="", 
            marker=".", 
            color="black",             
        )
        
        ax.set_ylabel(row_y_titles[row] if col == 0 else "", loc="center", fontsize=18)
        ax.ticklabel_format(axis="y", style="sci", scilimits=(0,0))

        # Plot PDG shape for comparison
        pdg_mass = 1.2295
        pdg_width = 0.142
        pdg_curve = d_wave_bw_model(function_masses, scale_opt, pdg_mass, pdg_width)
        ax.plot(function_masses, pdg_curve, color="blue", linestyle="--", linewidth=2, alpha=0.6, label="PDG BW")

        # calculate chi squared per degree of freedom for the fit
        residuals = fit_values - d_wave_bw_model(fit_masses, *popt)
        chi_squared = np.sum((residuals / fit_errors) ** 2)
        dof = len(fit_values) - len(popt)
        chi_squared_per_dof = chi_squared / dof
        
        if row == 0:
            t_low, t_high = results.get_t_edges()
            ax.set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=16)    
        if row == 2:
            ax.set_xlabel(r"$\omega\pi^0$ inv. mass (GeV)")
        
        textstr = (
            f"M: {round(mass_opt*1000)} ± {round(perr[1]*1000)} MeV\n"
            rf"$\Gamma$: {round(width_opt*1000)} ± {round(perr[2]*1000)} MeV"
            "\n"
            rf"$\chi^2$/df: {chi_squared_per_dof:.2f}"
        )
        ax.text(0.95, 0.95, textstr, transform=ax.transAxes, fontsize=12,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.savefig(f"{STUDY_DIR}/BW-b1_d_refl+_all_t_bins.pdf")

## Other
* Maybe look at phase differences between m projections of same jpl combo?

### Convergence Rates
View the rate of successful, failed, and converged but with bad error matrix fits

In [ ]:
fig, axs = plt.subplots(
    1,
    4,    
    sharey="row",         
    figsize=(18, 5),
    layout="constrained",
)
for col, (t_bin, results) in enumerate(binned_results.items()):
    my_kwargs = {
        "successful_kwargs": {"markersize":6},
        "failed_kwargs": {"markersize":6},
        "bad_matrix_kwargs": {"markersize":6}
    }
    results.plot.diagnostic.convergence_rates(ax=axs[col], **my_kwargs)
    t_low, t_high = results.get_t_edges()
    axs[col].set_title(rf"${t_low} \leq -t \leq {t_high}$ GeV$^2$", fontsize=16)
    if col != 0:
        axs[col].set_ylabel("")
        if axs[col].get_legend() is not None:
            axs[col].get_legend().remove()
plt.savefig(f"{STUDY_DIR}/convergence_rates_all_t_bins.pdf")

## Cleanup
zip together all the pdfs in each t bin, and zip the pdfs that describe all t bins together

In [ ]:
%%bash
cd ${STUDY_DIR}
zip -u all_t_bin_plots.zip *.pdf
zip -ru single_t_bin_plots.zip t_*/plots/*.pdf